Step 1: Prepare documents

In [1]:
import pandas as pd

df = pd.read_csv("../data/incidents_train.csv", index_col=False, dtype=str)
df = df.rename(columns={"Unnamed: 0": "id", "hazard-category": "hazard_category", "product-category": "product_category"})

Step 2: Create Embeddings using Pretrained Models

In [2]:
from sentence_transformers import SentenceTransformer

/home/codespace/.local/share/virtualenvs/food-hazard-detection-nVkya60Z/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/home/codespace/.local/share/virtualenvs/food-hazard-detection-nVkya60Z/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
v = model.encode('Which allergen do biscuits contain?')

In [5]:
len(v)

384

In [7]:
df.head()

,id,year,month,day,country,title,text,hazard_category,product_category,hazard,product
0,0,1994,1,7,us,Recall Notification: FSIS-024-94,Case Number: 024-94 \n Date Opene...,biological,"meat, egg and dairy products",listeria monocytogenes,smoked sausage
1,1,1994,3,10,us,Recall Notification: FSIS-033-94,Case Number: 033-94 \n Date Opene...,biological,"meat, egg and dairy products",listeria spp,sausage
2,2,1994,3,28,us,Recall Notification: FSIS-014-94,Case Number: 014-94 \n Date Opene...,biological,"meat, egg and dairy products",listeria monocytogenes,ham slices
3,3,1994,4,3,us,Recall Notification: FSIS-009-94,Case Number: 009-94 \n Date Opene...,foreign bodies,"meat, egg and dairy products",plastic fragment,thermal processed pork meat
4,4,1994,7,1,us,Recall Notification: FSIS-001-94,Case Number: 001-94 \n Date Opene...,foreign bodies,"meat, egg and dairy products",plastic fragment,chicken breast


In [9]:
documents = df.to_dict(orient='records')

In [10]:
documents[0]

{'id': '0',
 'year': '1994',
 'month': '1',
 'day': '7',
 'country': 'us',
 'title': 'Recall Notification: FSIS-024-94',
 'text': "Case Number: 024-94   \n            Date Opened: 07/01/1994   \n            Date Closed: 09/22/1994 \n    \n            Recall Class:  1   \n            Press Release (Y/N):  Y  \n    \n            Domestic Est. Number:  05893  P   \n              Name:  GERHARD'S NAPA VALLEY SAUSAGE\n    \n            Imported Product (Y/N):  N       \n            Foreign Estab. Number:  N/A\n    \n            City:  NAPA    \n            State:  CA   \n            Country:  USA\n    \n            Product:  SMOKED CHICKEN SAUSAGE\n    \n            Problem:  BACTERIA   \n            Description: LISTERIA\n    \n            Total Pounds Recalled:  2,894   \n            Pounds Recovered:  2,894",
 'hazard_category': 'biological',
 'product_category': 'meat, egg and dairy products',
 'hazard': 'listeria monocytogenes',
 'product': 'smoked sausage'}

In [11]:
from tqdm.auto import tqdm

In [12]:
#create the dense vector using the pre-trained model
operations = []
for doc in tqdm(documents):
    title = doc['title']
    hazard = doc['hazard'] + ' ' + doc['hazard_category']
    product = doc['product'] + ' ' + doc['product_category']
    doc_info = " ".join([title, hazard, product])

    #doc['title_vector'] = model.encode(title)
    #doc['hazard_vector'] = model.encode(hazard)
    #doc['product_vector'] = model.encode(product)
    doc['doc_vector'] = model.encode(doc_info)
    operations.append(doc)


  0%|          | 0/5082 [00:00<?, ?it/s]

In [13]:
documents[0]

{'id': '0',
 'year': '1994',
 'month': '1',
 'day': '7',
 'country': 'us',
 'title': 'Recall Notification: FSIS-024-94',
 'text': "Case Number: 024-94   \n            Date Opened: 07/01/1994   \n            Date Closed: 09/22/1994 \n    \n            Recall Class:  1   \n            Press Release (Y/N):  Y  \n    \n            Domestic Est. Number:  05893  P   \n              Name:  GERHARD'S NAPA VALLEY SAUSAGE\n    \n            Imported Product (Y/N):  N       \n            Foreign Estab. Number:  N/A\n    \n            City:  NAPA    \n            State:  CA   \n            Country:  USA\n    \n            Product:  SMOKED CHICKEN SAUSAGE\n    \n            Problem:  BACTERIA   \n            Description: LISTERIA\n    \n            Total Pounds Recalled:  2,894   \n            Pounds Recovered:  2,894",
 'hazard_category': 'biological',
 'product_category': 'meat, egg and dairy products',
 'hazard': 'listeria monocytogenes',
 'product': 'smoked sausage',
 'doc_vector': array([-3.74

Step 3: Setup ElasticSearch connection

In [16]:
!pip install elasticsearch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.6/524.6 kB 5.4 MB/s eta 0:00:00


In [17]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://127.0.0.1:9200') 

es_client.info()

ObjectApiResponse({'name': 'd608436b3a40', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'UB-7xabhRGq_DVUNcNIGOA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

Step 4: Create Mappings and Index

In [18]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "title": {"type": "text"},
            "hazard_category": {"type": "text"},
            "product_category": {"type": "text"},
            "hazard": {"type": "text"},
            "product": {"type": "text"} ,
            "doc_vector": {"type": "dense_vector", "dims": 384, "index": True, "similarity": "cosine"},
        }
    }
}

In [19]:
index_name = "food-hazard-detection"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'food-hazard-detection'})

Step 5: Add documents into index

In [20]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [22]:
query='Which allergen do biscuits contain?'

In [23]:
v_q = model.encode(query)

In [26]:
query = {
    "field": "doc_vector",
    "query_vector": v_q,
    "k": 5,
    "num_candidates": 10000, 
}

In [27]:
res = es_client.search(index=index_name, knn=query, source=["title", "hazard_category", "product_category", "hazard", "product"])
res["hits"]["hits"]

[{'_index': 'food-hazard-detection',
  '_id': 'fZgyZpIB94Ty_xnEm8-q',
  '_score': 0.86106145,
  '_ignored': ['text.keyword'],
  '_source': {'product': 'biscuits',
   'hazard': 'peanuts and products thereof',
   'hazard_category': 'allergens',
   'title': 'Coles—Chewy Oat and Raisin and Anzac Cookies',
   'product_category': 'cereals and bakery products'}},
 {'_index': 'food-hazard-detection',
  '_id': '95g0ZpIB94Ty_xnET-I9',
  '_score': 0.8405731,
  '_ignored': ['text.keyword'],
  '_source': {'product': 'biscuits',
   'hazard': 'peanuts and products thereof',
   'hazard_category': 'allergens',
   'title': 'Imported biscuit may contain allergen (peanuts)',
   'product_category': 'cereals and bakery products'}},
 {'_index': 'food-hazard-detection',
  '_id': 'b5gzZpIB94Ty_xnEl9rg',
  '_score': 0.83848,
  '_ignored': ['text.keyword'],
  '_source': {'product': 'biscuits',
   'hazard': 'peanuts and products thereof',
   'hazard_category': 'allergens',
   'title': 'Deli Spice Sesame Biscuit B

In [42]:
def elastic_search_knn(field, vector):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000
    }

    search_query = {
        "knn": knn,
        "_source": ["title", "hazard_category", "product_category", "hazard", "product", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [29]:
def doc_vector_knn(q):
    question = q['question']

    v_q = model.encode(question)

    return elastic_search_knn('doc_vector', v_q)

In [31]:
df_ground_truth = pd.read_csv('../data/ground-truth-retrieval.tsv', delimiter="\t")

In [32]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [33]:
ground_truth[0]

{'id': 1013,
 'question': 'What specific batches of vacuum-packed Organic Tofu were recalled by The Engine Shed?'}

In [34]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [35]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [53]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q)
        relevance = [int(d['id']) == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [54]:
evaluate(ground_truth, doc_vector_knn)

  0%|          | 0/2335 [00:00<?, ?it/s]

{'hit_rate': 0.4668094218415418, 'mrr': 0.37291220556745225}